In [1]:
import os
import numpy as np
import pandas as pd
from scipy import signal
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
import joblib

In [2]:
# ---------- config ----------
DYS_CSV = "database/dysgraphia.csv"
NORM_CSV = "database/normal.csv"
WINDOW_SEC = 2.0
WINDOW_STEP_SEC = 1.0  # overlap = WINDOW_SEC - WINDOW_STEP_SEC
MODEL_OUT = "models/dysgraphia_rf_pipeline.pkl"
RANDOM_STATE = 42

In [3]:
os.makedirs("models", exist_ok=True)

def load_csv(path):
    df = pd.read_csv(path)
    # ensure expected columns exist
    expected = ["timestamp","ax","ay","az","gx","gy","gz","p1","p2"]
    missing = [c for c in expected if c not in df.columns]
    if missing:
        raise ValueError(f"Missing columns in {path}: {missing}")
    # normalize timestamp to seconds (if in ms, convert)
    ts = df['timestamp'].values
    # if timestamps are large (ms), convert:
    if ts.mean() > 1e6:
        df['timestamp'] = df['timestamp'] / 1000.0
    return df[expected].reset_index(drop=True)

In [4]:

# def estimate_fs(df):
#     ts = df['timestamp'].values
#     diffs = np.diff(ts)
#     median_dt = np.median(diffs)
#     if median_dt == 0:
#         # fallback to 50 Hz
#         return 50.0
#     return 1.0/median_dt

def estimate_fs(df):
    """Estimate sampling frequency from timestamp deltas."""
    ts = df['timestamp'].values
    diffs = np.diff(ts)
    diffs = diffs[diffs > 0]
    if len(diffs) == 0:
        return 45.0  # fallback
    median_dt_ms = np.median(diffs)  # in ms (since 22 ms per sample)
    fs = 1000.0 / median_dt_ms
    return fs

In [5]:
# def sliding_windows(df, window_sec, step_sec):
#     fs = estimate_fs(df)
#     window_len = int(round(window_sec * fs))
#     step = int(round(step_sec * fs))
#     array = df[['ax','ay','az','gx','gy','gz','p1','p2']].values
#     N = array.shape[0]
#     idx = 0
#     while idx + window_len <= N:
#         yield array[idx: idx+window_len]
#         idx += step

def sliding_windows(df, window_sec, step_sec):
    fs = estimate_fs(df)
    window_len = int(round(window_sec * fs))
    step = int(round(step_sec * fs))
    array = df[['ax','ay','az','gx','gy','gz','p1','p2']].values
    N = array.shape[0]

    if N < window_len:
        print(f"⚠️ Skipping file: only {N} samples < {window_len} required for one window.")
        return

    idx = 0
    while idx + window_len <= N:
        yield array[idx: idx + window_len]
        idx += step

In [6]:
# feature extraction per window
def extract_features(win, fs):
    # win: (T, 8) columns ax..p2
    features = []
    # statistical features for each channel
    ch_names = ['ax','ay','az','gx','gy','gz','p1','p2']
    for i in range(win.shape[1]):
        x = win[:, i]
        features += [
            x.mean(),
            x.std(),
            np.min(x),
            np.max(x),
            np.percentile(x, 25),
            np.percentile(x, 50),
            np.percentile(x, 75),
            np.mean(np.abs(np.diff(x))),  # mean absolute derivative
        ]
    # magnitude features for accelerometer and gyroscope
    acc = np.linalg.norm(win[:, 0:3], axis=1)
    gyr = np.linalg.norm(win[:, 3:6], axis=1)
    features += [
        acc.mean(), acc.std(), acc.max(),
        gyr.mean(), gyr.std(), gyr.max()
    ]
    # frequency-domain: dominant freq for each accel channel
    for i in range(3):  # ax ay az
        x = win[:, i]
        # remove mean
        x = x - x.mean()
        # compute PSD via Welch
        f, Pxx = signal.welch(x, fs=fs, nperseg=min(len(x), 256))
        # dominant freq
        if Pxx.size>0:
            dom = f[np.argmax(Pxx)]
            bw = np.sum(Pxx > (Pxx.max() * 0.5))  # crude bandwidth proxy
        else:
            dom = 0.0
            bw = 0.0
        features += [dom, bw]
    return np.array(features)

In [7]:
# def build_dataset(df_list, labels, window_sec, step_sec):
#     X = []
#     y = []
#     for df, lbl in zip(df_list, labels):
#         fs = estimate_fs(df)
#         for win in sliding_windows(df, window_sec, step_sec):
#             feat = extract_features(win, fs)
#             X.append(feat)
#             y.append(lbl)
#     X = np.vstack(X)
#     y = np.array(y)
#     return X, y


def build_dataset(df_list, labels, window_sec, step_sec):
    X = []
    y = []
    for df, lbl in zip(df_list, labels):
        fs = estimate_fs(df)
        count = 0
        for win in sliding_windows(df, window_sec, step_sec):
            if len(win) == 0:
                continue
            feat = extract_features(win, fs)
            X.append(feat)
            y.append(lbl)
            count += 1
        print(f"✅ Generated {count} windows for label {lbl} ({'Dysgraphia' if lbl==1 else 'Normal'})")
    if not X:
        raise ValueError("No feature windows generated. Try lowering WINDOW_SEC or verify timestamps.")
    X = np.vstack(X)
    y = np.array(y)
    return X, y

In [8]:
def main():
    print("Loading CSVs...")
    dys = load_csv(DYS_CSV)
    norm = load_csv(NORM_CSV)
    print(f"dys samples: {len(dys)}, normal samples: {len(norm)}")

    print("Building dataset (windowing & features)...")
    X, y = build_dataset([dys, norm], [1, 0], WINDOW_SEC, WINDOW_STEP_SEC)
    print("Feature matrix shape:", X.shape)

    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, stratify=y, random_state=RANDOM_STATE)

    # pipeline with scaler + classifier
    pipe = Pipeline([
        ('scaler', StandardScaler()),
        ('clf', RandomForestClassifier(random_state=RANDOM_STATE, n_jobs=-1))
    ])

    # quick grid search (optional; you can skip or reduce)
    param_grid = {
        'clf__n_estimators': [100],
        'clf__max_depth': [10, 20, None],
        'clf__min_samples_split': [2,5]
    }

    print("Training model (GridSearchCV)...")
    grid = GridSearchCV(pipe, param_grid, cv=3, scoring='accuracy', n_jobs=-1, verbose=1)
    grid.fit(X_train, y_train)

    print("Best params:", grid.best_params_)
    best = grid.best_estimator_

    y_pred = best.predict(X_test)
    print("Test accuracy:", accuracy_score(y_test, y_pred))
    print("Classification report:\n", classification_report(y_test, y_pred))
    print("Confusion matrix:\n", confusion_matrix(y_test, y_pred))

    # save the pipeline
    joblib.dump(best, MODEL_OUT)
    print(f"Saved model pipeline to {MODEL_OUT}")

In [9]:
if __name__ == "__main__":
    main()

Loading CSVs...
dys samples: 1647, normal samples: 1317
Building dataset (windowing & features)...
✅ Generated 35 windows for label 1 (Dysgraphia)
✅ Generated 28 windows for label 0 (Normal)
Feature matrix shape: (63, 76)
Training model (GridSearchCV)...
Fitting 3 folds for each of 6 candidates, totalling 18 fits
Best params: {'clf__max_depth': 10, 'clf__min_samples_split': 2, 'clf__n_estimators': 100}
Test accuracy: 1.0
Classification report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00         6
           1       1.00      1.00      1.00         7

    accuracy                           1.00        13
   macro avg       1.00      1.00      1.00        13
weighted avg       1.00      1.00      1.00        13

Confusion matrix:
 [[6 0]
 [0 7]]
Saved model pipeline to models/dysgraphia_rf_pipeline.pkl


In [ ]:
import pandas as pd

dys = pd.read_csv("database/dysgraphia.csv")
norm = pd.read_csv("database/normal.csv")

print("Dysgraphia rows:", len(dys))
print("Normal rows:", len(norm))
print("Dysgraphia columns:", list(dys.columns))
print("Normal columns:", list(norm.columns))
print(dys.head(3))

Dysgraphia rows: 1647
Normal rows: 1317
Dysgraphia columns: ['timestamp', 'ax', 'ay', 'az', 'gx', 'gy', 'gz', 'p1', 'p2']
Normal columns: ['timestamp', 'ax', 'ay', 'az', 'gx', 'gy', 'gz', 'p1', 'p2']
   timestamp    ax    ay    az    gx    gy    gz  p1  p2
0          0  8.52  5.84 -3.26  0.15  0.23  0.10  21  15
1         22  8.32  5.73 -3.38  0.25  0.28  0.10  21   3
2         44  6.93  5.89 -4.77 -0.10  0.70  0.79  21  16


: 